# Manipulation des zones géographiques.

Les zones géographiques sont des "rectangles" déterminés par 4 coordonnées en latitude et longitude. On va commencer par créer de telles zones, cela nous permettra par la suite de les extraire des images téléchargées.

_Version 1.0 [31/01/2023] Jérôme Lacaille_

In [1]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.rcParams["figure.figsize"] = (9,6) # Pour l'affichage d'images un peu plus grandes.

Vous trouverez ci-dessous un lien vers la doc de paramétrage de "matplotlib.

> [Customizing Matplotlib with style sheets and rcParams](https://matplotlib.org/stable/tutorials/introductory/customizing.html)

In [2]:
# Ce bout de code est pratique car il permet de recharger automatiquement un package quand vous le développez.
%reload_ext autoreload
%autoreload 2

## (Digression) Les fonction magiques.

Jupyter-Notebooks dispose de macros pratiques comme %matplotlib ou %autoreload. Consultez la doc.

In [4]:
%magic

## 1. Gestion des zones géographiques.

Nous allons d'abord créer des fichiers de zones géographiques correspondant à des "rectangles" en latitude et longitudes.
Ce que l'on souhaite c'est pouvoir manipuler des dictionnaires avec deux composantes : un nom et les coordonnées du rectangle :

    {
        'name' : "Le nom de la zone",
        'bbox' : [lat_min, lon_min, lat_max, lon_max]
    }

Ensuite on va récupérer automatiquement ces rectangles depuis un site spécialisé et créer un nouvel objet Python capable de gérer ces listes.

## (Digression) Le Markdown.

Les cellules textes sont écrites en <mark>langage Markdown</mark> qui simplifie le HTML et inclue des fonctionnalités LaTeX.

> [Guide MarkDown](https://www.markdownguide.org/)

### 1.1 Etude d'une zone géographique.
En utilisant le site [geozone.io](https://geojson.io/) on peut créer facilement des GEOFILES qui contiennet des informations sur la latitude et la longitude de zones rectangulaires.

Un bouton vous permet de copier facilement un descriptif json (geojson en fait) et le copier directement dans une cellule comme un dictionnaire.

In [5]:
geofile = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {'name' : 'Villaroche'},
      "geometry": {
        "coordinates": [
          [
            [
              2.6255314173795625,
              48.62074698105093
            ],
            [
              2.6255314173795625,
              48.57874855877512
            ],
            [
              2.7130686835237725,
              48.57874855877512
            ],
            [
              2.7130686835237725,
              48.62074698105093
            ],
            [
              2.6255314173795625,
              48.62074698105093
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [6]:
geofile

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Villaroche'},
   'geometry': {'coordinates': [[[2.6255314173795625, 48.62074698105093],
      [2.6255314173795625, 48.57874855877512],
      [2.7130686835237725, 48.57874855877512],
      [2.7130686835237725, 48.62074698105093],
      [2.6255314173795625, 48.62074698105093]]],
    'type': 'Polygon'}}]}

In [7]:
type(geofile)

dict

In [8]:
geofile['features'][0]['geometry']['coordinates']

[[[2.6255314173795625, 48.62074698105093],
  [2.6255314173795625, 48.57874855877512],
  [2.7130686835237725, 48.57874855877512],
  [2.7130686835237725, 48.62074698105093],
  [2.6255314173795625, 48.62074698105093]]]

Pour mieux gérer vos zones géographiques (geozones) je vous propose de rajouter un champ 'name' aux propriétés. Vous verrez que cela nous facilitera le travail par la suite.

          "properties": {'name' : 'Villaroche'},

In [9]:
# On va avoir besoin de numpy pour gérer les coordinnées.
import numpy as np

On va maintenant récupérer le nom et les coordonnées.

In [10]:
name = geofile['features'][0]['properties']['name']
name

'Villaroche'

In [11]:
G = np.array(geofile['features'][0]['geometry']['coordinates'][0])
G

array([[ 2.62553142, 48.62074698],
       [ 2.62553142, 48.57874856],
       [ 2.71306868, 48.57874856],
       [ 2.71306868, 48.62074698],
       [ 2.62553142, 48.62074698]])

On va maintenant chercher les coins de notre rectangle. Attention à ne pas intervertir longitude et latitude. Pour cela je vous propose de regarder un rectangle que vous caisssez bien, avec l'Ile de France par exemple, on ne peut pas se tromper.

In [12]:
lon0, lat0 = G.min(axis=0)
lat0, lon0

(48.57874855877512, 2.6255314173795625)

In [13]:
lon1, lat1 = G.max(axis=0)
lat1, lon1

(48.62074698105093, 2.7130686835237725)

In [14]:
G.min(axis=0)

array([ 2.62553142, 48.57874856])

Finalement vous pouvez créer votre zone comme un dictionnaire.

In [15]:
zone = dict(name=geofile['features'][0]['properties']['name'], bbox=[lat0, lon0, lat1, lon1])
zone

{'name': 'Villaroche',
 'bbox': [48.57874855877512,
  2.6255314173795625,
  48.62074698105093,
  2.7130686835237725]}

### 1.2 Création d'une première fonction.

Le code que nous venons de créer peut s'automatiser. On va créer une fonction `getzone()` qui va prendre l'objet copié depuis geojson.io et renvoie la zone en question.

    def getzone(geofile):
        ...
        return zone

Cette fonction va être placée dans le module "geozone.py" mais vous pouvez déjà la mettre en oeuvre interactivement dans ce notebook.

In [16]:
f"Geozone [{lat0:.1f},{lon0:.1f},{lat1:.1f},{lon1:.1f}]"

'Geozone [48.6,2.6,48.6,2.7]'

In [17]:
"Geozone [{:.1f},{:.1f},{},{:.1f}]".format(lat0,lon0,lat1,lon1)

'Geozone [48.6,2.6,48.62074698105093,2.7]'

In [18]:
def getzone(geolist):
    
    G0 = geolist['features'][0]

    G = np.array(G0['geometry']['coordinates'][0])
    lon0,lat0 = G.min(axis=0)
    lon1,lat1 = G.max(axis=0)

    # Récupération du nom de la zone.
    if 'name' in G0['properties']:
        name = G0['properties']['name']
    else:
        name = f"Geozone [{lat0:.1f},{lon0:.1f},{lat1:.1f},{lon1:.1f}]"

    zone = dict(
        name = name,
        bbox = [lat0, lon0, lat1, lon1]
    )
    return zone

In [19]:
getzone

<function __main__.getzone(geolist)>

Maintenant il faut tester si cette fonction marche bien.

In [20]:
getzone(geofile)

{'name': 'Villaroche',
 'bbox': [48.57874855877512,
  2.6255314173795625,
  48.62074698105093,
  2.7130686835237725]}

Vous recopirez cette fonction au module "geozone" en rajoutant un paramètre `i` supplémentaire pour aller chercher chacune des zones de la liste fournie en argument.

    def getzone(geolist, i=0):
        ...

On force la valeur du paramètre `i` à 0, comme ça on peut toujours utiliser la fonction avec un seul argument.

<mark>N'oubliez pas d'importer le package.</mark>

In [21]:
# On importe la package satellite.
import satellite as sat

Pour que le package soit reconnu, il faut que le dossier "satellite" se trouve dans un répertoire pointé par la variable d'environnement PYTHONPATH. Dans mon cas il s'agit ddu répertoire "/Users/holie/wrk0".

In [25]:
!echo $PYTHONPATH

/Users/holie/wrk:/Users/holie/wrk0


In [26]:
# On teste la fonction programmée dans le package.
sat.geozone.getzone(geofile)

{'name': 'Villaroche',
 'bbox': [48.57874855877512,
  2.6255314173795625,
  48.62074698105093,
  2.7130686835237725]}

Pour que la fonction soit plus facile d'accès on rajoute `getzone` à l'import de "geozone" dans "__init__.py".

    from .geozone import getzone, EARTHDIR

In [27]:
# On teste notre facilité d'import.
sat.getzone(geofile)

{'name': 'Villaroche',
 'bbox': [48.57874855877512,
  2.6255314173795625,
  48.62074698105093,
  2.7130686835237725]}

### 1.3 Gestion des erreurs.

Quand on écrit un package, il faut faire attention à ne pas risquer de produire des erreurs incontrôlées. On va donc ajouter quelques éléments de gestion d'erreur à notre fonction.

Python gère les erreurs à l'aide d'exceptions. Plusieurs types d'exceptions sont disponibles (voir la documentation de Python sur les <mark>Exceptions</mark>)

> [Built-in Exceptions](https://docs.python.org/3/library/exceptions.html#base-classes)

Il est possible de créer des exceptions en exécutant par exemple un appel à l'instruction `assert`, mais cela génère une `AssertionError` qui reste assez générique. On peut aussi vouloir créer sa propre execption et dans ce cas on va dériver une classe exception et fabriquer la notre.

Dans notre cas on va créer une classe dérivée de `ValueError` qui est une exception assez standard quand on a un problème du aux données d'entrée d'une fonction.

Vous rajouterez la création d'une classe spécifique `GeofileException` au module geozone qui dérive de `ValueError`. Comme cette Exception se comporte exactement comme `ValueError` il n'y a rien d'autre à faire qu'à la déclarer. Quand elle sera lancée on verra tout de suite si l'erreur vient de l'objet.

Pour exécuter une exception on utilise la commande `raise`.

In [28]:
raise sat.geozone.GeofileException("Un essai.")

GeofileException: Un essai.

Nous utilisezons cette exception pour tester l'existance des clés 'properties', 'geometry' et 'coordinates' dans le paramètre envoyé en entrée.

Ci-dessous j'ai remplacé la clé 'geometry' par 'geometrie', et je teste si mon code réagit bien.

In [29]:
badgeofile = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {'name' : 'Chypre'},
      "geometrie": {
        "coordinates": [
          [
            [
              23.554712962096346,
              35.607613541778534
            ],
            [
              23.554712962096346,
              34.89248636947745
            ],
            [
              26.302614086102693,
              34.89248636947745
            ],
            [
              26.302614086102693,
              35.607613541778534
            ],
            [
              23.554712962096346,
              35.607613541778534
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

In [30]:
sat.getzone(badgeofile)

GeofileException: No geometry in geolist[0].

C'est plus clair ! On voit tout de suite que c'est un problème d'entrée.

Si on veut tester qu'un argument entier est bien dans les bornes attendues, ce que l'on fait en général c'es appeler directement `assert()`.
C'est ce que l'on va faire pour vérifier si l'entier passé est bien entre 0 et la longeur de la liste de zones géographiques donnés en entrée.

In [31]:
sat.getzone(geofile,-1)

AssertionError: Le numéro de zone doit être positif.

In [32]:
sat.getzone(geofile,3)

AssertionError: La liste ne contient que 1 zone.

**Astuce** : J'ai utilisé l'opérateur ternaire pour m'assure que "zone" sera au singulier si le nombres de zones est inféreiyer ou égal à 1 et au pluriel sinon.

    n = len(geolist['features'])
    sz = "zones" if n>1 else "zone"

**Et si on avait oublié le nom ?**
Vous vous souvenez que j'avais proposé de rajouter le nom de votre zone dans le champ "properties". On ne va toute de même pas créer une erreur à cause de cela, je vous propose dans le cas d'un nom absent de créer un texte à partir des coordonnées.


In [33]:
notnamedgeofile = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              23.554712962096346,
              35.607613541778534
            ],
            [
              23.554712962096346,
              34.89248636947745
            ],
            [
              26.302614086102693,
              34.89248636947745
            ],
            [
              26.302614086102693,
              35.607613541778534
            ],
            [
              23.554712962096346,
              35.607613541778534
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

On essaue de se limiter à une seule valeur après la virgule.

In [34]:
sat.getzone(notnamedgeofile)

{'name': 'Geozone [34.9,23.6,35.6,26.3]',
 'bbox': [34.89248636947745,
  23.554712962096346,
  35.607613541778534,
  26.302614086102693]}

### 1.4 Gestion d'une liste de zones géographiques.

Maintenant nous allons profiter d'une fonction sympatique de [geozone.io](https://geojson.io/) qui nous permet de sélectionner en une seule fois plusieurs zones. Une fois que c'est fait enregistrez le fichier. Par défaut, il s'appelle "map.geojson".

Je vous propose de placer ce fichier dans le sous-répertoire pointé par EARTHDIR.

Créez cette fois une nouvelle fonction `zonelist()` qui renvoie la liste de toutes les zones du fichier. Par défaut, cette fonction av chercher le fichier dans EARTHDIR, et prend "map.geojson" comme valeur par défaut.

    def zonelist(geomap  "map.geojson")

Pour commencer il faut convertir le fichier "map.geojson" en un dictionnaire. Le plus simple est d'utiliser le module json de Python.

In [35]:
import os,json

In [36]:
geomapfile = os.path.join(sat.EARTHDIR,'map.geojson')
geomapfile

'/Users/holie/wrk0/satellite/data/zones/_Earth/map.geojson'

Je préfère toujours utiliser l'instruction `with` pour gerer proprement les ressoures.

In [38]:
with open(geomapfile) as g:
    geolist = json.load(g)
geolist

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Ile de France'},
   'geometry': {'coordinates': [[[2.0085203859495095, 48.99193420772434],
      [2.0085203859495095, 48.7535883521073],
      [2.6788390690815334, 48.7535883521073],
      [2.6788390690815334, 48.99193420772434],
      [2.0085203859495095, 48.99193420772434]]],
    'type': 'Polygon'}},
  {'type': 'Feature',
   'properties': {'name': 'Barcelone'},
   'geometry': {'coordinates': [[[1.6649881237897546, 41.63394152808198],
      [1.6649881237897546, 41.22007856049126],
      [2.4690345841926273, 41.22007856049126],
      [2.4690345841926273, 41.63394152808198],
      [1.6649881237897546, 41.63394152808198]]],
    'type': 'Polygon'}},
  {'type': 'Feature',
   'properties': {'name': 'Crète'},
   'geometry': {'coordinates': [[[23.397028867484522, 35.631415093017225],
      [23.397028867484522, 34.85709260445668],
      [26.44578481885614, 34.85709260445668],
      [26.44578481885614, 35

Voila ! Vous avez ce qu'il faut pour rajouter `zonelist()` à "geozone". N'oubliez pas la gestion des erreurs.

In [39]:
Z = sat.zonelist()
Z

[{'name': 'Ile de France',
  'bbox': [48.7535883521073,
   2.0085203859495095,
   48.99193420772434,
   2.6788390690815334]},
 {'name': 'Barcelone',
  'bbox': [41.22007856049126,
   1.6649881237897546,
   41.63394152808198,
   2.4690345841926273]},
 {'name': 'Crète',
  'bbox': [34.85709260445668,
   23.397028867484522,
   35.631415093017225,
   26.44578481885614]}]

## 2. Construction d'une classe

Maintenant nous allons regrouper tout ce que nous avons appris pour faire une classe `GeoZone` initialisée avec un gichier "map.geojson".

Cette classe doit avoir les fonctions suivante.

* S'initialiser par "map.gojson".

        gz = Geozone()

* Renvoyer sa longeur
    
        len(gz)

* Iterer sur les zones
    
        for z in gz:
            ...

* Accéder à une zone par indexation.
    
        z = gz[2]

N'oubliez pas de rajouter GoZone dans le module `__init__.py`.


In [40]:
gz = sat.GeoZone()

In [41]:
len(gz)

3

In [42]:
for z in gz:
    print(z['name'])
    

Ile de France
Barcelone
Crète


In [43]:
[z['name'] for z in gz]

['Ile de France', 'Barcelone', 'Crète']

In [44]:
gz[2]

{'name': 'Crète',
 'bbox': [34.85709260445668,
  23.397028867484522,
  35.631415093017225,
  26.44578481885614]}

In [45]:
try:
    gz[5]
except Exception as e:
    print(type(e))
    print(e.args)
else:
    print("OK")
finally:
    print("On continue...")

<class 'satellite.geozone.GeofileException'>
('Zone inexistante, doit etre comprise entre entre 0 et 2.',)
On continue...


In [46]:
print(gz.__doc__)

 GEOZONE - Un objet contenant les zones géographiques et permettant de les manipuler.

    GeoZone est initié à partir d'un fichier issu de geojson.io. Par défaut, ce fichier est "map.geojson" et est situé dans le sous-répertoire "data/_Earth".
    


Faites en sorte que l'on puisse aussi créer des GeoZones en ligne à partir d'un dictionnaire.

In [47]:
# Astuce :
isinstance(z,dict)

True

In [48]:
gz2 = sat.GeoZone(geofile)

In [49]:
len(gz2)

1

In [50]:
gz2[0]

{'name': 'Villaroche',
 'bbox': [48.57874855877512,
  2.6255314173795625,
  48.62074698105093,
  2.7130686835237725]}

_Fini_